In [1]:
using CompTime, InteractiveUtils

Polynomial:

$p(x) = a_1 + a_2 x + \cdots + a_n x^{n-1}$


Horner's method:

$p(x) = (((a_n x  + a_{n-1}) x + a_{n-2}) x + \cdots + a_{2})x + a_{1}$


In [2]:
"""
Evaluate a polynomial whose coefficients are given in ascending
order in `c`, at the point `x`, using Horner's rule.
"""

function horner(a, x)
    n = length(a)
    y = a[n]

    for k in n-1:-1:1
        y = x * y + a[k]
    end

    return y
end

horner (generic function with 1 method)

In [3]:
a = [-1, 3, -3, 1]

4-element Vector{Int64}:
 -1
  3
 -3
  1

In [4]:
horner(a, 10)

729

In [5]:
@code_lowered horner(a, 10)

CodeInfo(
1 ─       n = Main.length(a)
│         y = Base.getindex(a, n)
│   %3  = n - 1
│   %4  = %3:-1:1
│         @_4 = Base.iterate(%4)
│   %6  = @_4 === nothing
│   %7  = Base.not_int(%6)
└──       goto #4 if not %7
2 ┄ %9  = @_4
│         k = Core.getfield(%9, 1)
│   %11 = Core.getfield(%9, 2)
│   %12 = x * y
│   %13 = Base.getindex(a, k)
│         y = %12 + %13
│         @_4 = Base.iterate(%4, %11)
│   %16 = @_4 === nothing
│   %17 = Base.not_int(%16)
└──       goto #4 if not %17
3 ─       goto #2
4 ┄       return y
)

In [6]:
function horner_tuple(c::NTuple{N,T}, x::T) where {N,T}
    y = c[N]

    for k in N-1:-1:1
        y = x * y + c[k]
    end

    return y
end

horner_tuple (generic function with 1 method)

In [7]:
t = (-1, 3, -3, 1)

(-1, 3, -3, 1)

In [8]:
horner_tuple(t, 10)

729

In [9]:
@code_lowered horner_tuple(t, 10)

CodeInfo(
1 ─       y = Base.getindex(c, $(Expr(:static_parameter, 1)))
│   %2  = $(Expr(:static_parameter, 1)) - 1
│   %3  = %2:-1:1
│         @_4 = Base.iterate(%3)
│   %5  = @_4 === nothing
│   %6  = Base.not_int(%5)
└──       goto #4 if not %6
2 ┄ %8  = @_4
│         k = Core.getfield(%8, 1)
│   %10 = Core.getfield(%8, 2)
│   %11 = x * y
│   %12 = Base.getindex(c, k)
│         y = %11 + %12
│         @_4 = Base.iterate(%3, %10)
│   %15 = @_4 === nothing
│   %16 = Base.not_int(%15)
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return y
)

In [10]:
@generated function horner_gen(c::NTuple{N,T}, x::T) where {N,T}
    y = :(c[$N])

    for k in N-1:-1:1
        y = :(x * $y + c[$k])
    end

    return y
end

horner_gen (generic function with 1 method)

In [11]:
horner_gen(t, 10)

729

In [12]:
@code_lowered horner_gen(t, 10)

CodeInfo(
    @ /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb:1 within `horner_gen`
   ┌ @ /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb within `macro expansion`
1 ─│ %1  = Base.getindex(c, 4)
│  │ %2  = x * %1
│  │ %3  = Base.getindex(c, 3)
│  │ %4  = %2 + %3
│  │ %5  = x * %4
│  │ %6  = Base.getindex(c, 2)
│  │ %7  = %5 + %6
│  │ %8  = x * %7
│  │ %9  = Base.getindex(c, 1)
│  │ %10 = %8 + %9
└──│       return %10
   └
)

In [13]:
@ct_enable function horner_ct(c::NTuple{N,T}, x::T) where {N,T}
    y = c[N]

    @ct_ctrl for k in N-1:-1:1
        y = x * y + c[@ct(k)]
    end

    return y
end

runtime (generic function with 1 method)

In [14]:
horner_ct(t, 10)

729

In [15]:
@code_lowered comptime(horner_ct, t, 10)

CodeInfo(
    @ /home/roman/.julia/packages/CompTime/Ppb3B/src/CompTime.jl:137 within `comptime`
   ┌ @ /home/roman/.julia/packages/CompTime/Ppb3B/src/CompTime.jl:137 within `macro expansion` @ /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb:2
1 ─│       y = Base.getindex(c, $(Expr(:static_parameter, 1)))
│  │ @ /home/roman/.julia/packages/CompTime/Ppb3B/src/CompTime.jl:137 within `macro expansion` @ /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb:5
│  │ %2  = x * y
│  │ %3  = Base.getindex(c, 3)
│  │       y = %2 + %3
│  │ %5  = x * y
│  │ %6  = Base.getindex(c, 2)
│  │       y = %5 + %6
│  │ %8  = x * y
│  │ %9  = Base.getindex(c, 1)
│  │       y = %8 + %9
│  │ @ /home/roman/.julia/packages/CompTime/Ppb3B/src/CompTime.jl:137 within `macro expansion` @ /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb:8
│  │ %11 = y
│  │ @ /home/roman/.julia/packages/CompTime/Ppb3B/src/CompTime.jl:137 within `macro expansio

In [16]:
debug(horner_ct, t, 10)

quote
    #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb:1 =#
    #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb:2 =#
    y = c[N]
    #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb:4 =#
    begin
        begin
            #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb:5 =#
            y = x * y + c[3]
            #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb:6 =#
        end
        begin
            #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb:5 =#
            y = x * y + c[2]
            #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb:6 =#
        end
        begin
            #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horner.ipynb:5 =#
            y = x * y + c[1]
            #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/horne